In [1]:
import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,5]
inputedges = [(0,2),(4,5),(1,3),(0,4),(4,1),(2,5),(5,3)]

non_member = [4,5]
source = [0,1]
recievers = [[3],[2]]

sessions = [0,1]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


# ----- building LP ---------------

fstar_constraints = []
flow_constraints = []
capacity_constraints = []
variable_constraints = []


phi = {(i,j): cp.Variable() for j in recievers for i in sessions}   # f* == incoming reciever flow

fstar_constraints = [(-1 + sum(phi[i,j] for j in recievers for i in sessions)) == 0]

mu = {(i, a): cp.Variable() for a in edges for i in recievers}   # fia <= c(a)
xi = {(i, a): cp.Variable() for a in edges for i in recievers}   # fia <= c(a)

for i in recievers:
    for a in edges:
        variable_constraints += [xi[i,a] >= 0]

rho = {(i, v): cp.Variable() for v in non_member for i in recievers}   # outfrom non-member == into non-memebr
sigma = {(i): cp.Variable() for i in recievers}   # into source == 0
tau = {(i): cp.Variable() for i in recievers}   # outfrom reciever == 0




for i in recievers:
    for a in edges:
        equation = - mu[i,a] + xi[i,a]
        if a[1] in non_member:
            equation += rho[i,a[1]]
        if a[0] in non_member:
            equation -= rho[i,a[0]]
        if a[1] == int(source):
            equation += sigma[i]
        if a[0] == int(i):
            equation += tau[i]
        if a[1] == int(i):
            equation -= phi[i]
        
        variable_constraints += [mu[i,a] >= 0]
        flow_constraints += [equation == 0]
        

lambd = {a: cp.Variable() for a in edges}   

nu = {a1: cp.Variable() for a1 in undirected_edges}   # fia <= c(a)


for a1 in undirected_edges:
    a2 = (a1[1], a1[0]) 
    equation1 = nu[a1]
    equation2 = nu[a1]
    for i in recievers:
        equation1 -= (xi[i, a1] + lambd[a1])
        equation2 -= (xi[i, a2] + lambd[a2])

    variable_constraints += [lambd[a1] >= 0]
    variable_constraints += [lambd[a2] >= 0]

    capacity_constraints += [equation1 == 0]
    capacity_constraints += [equation2 == 0]
    

objective = cp.Minimize(sum(nu[a1] for a1 in undirected_edges))

constraints = fstar_constraints + flow_constraints + capacity_constraints + variable_constraints

problem = cp.Problem(objective, constraints)
problem.solve()

print("minimal objective:", sum(nu[a1] for a1 in undirected_edges).value)



print(xi[1,(3,1)].value)
print(xi[2,(2,6)].value)     #(5,2)is largest, and then (6,2)


[0 1 2 3 4 5]
{(1, 3), (4, 0), (0, 4), (3, 1), (5, 4), (2, 0), (1, 4), (0, 2), (4, 5), (5, 3), (2, 5), (4, 1), (3, 5), (5, 2)}


TypeError: unhashable type: 'list'